In [1]:
import pandas as pd
import warnings

In [2]:
# Para ignorar todos los warnings
warnings.filterwarnings("ignore")

Este notebook se centra la preparación de los datos para los distintos análisis.  
Primero nos quedaremos con las filas (en ambos datasets) cuyos identificadores de paciente coinciden. Estas son las observaciones que trataremos en nuestro estdio. Además, la base de datos clínica cuenta con algunos datos faltantes, los cuales vamos a imputar. 

In [3]:
# Leer el DataFrame de datos radiómicos 
radiomicas = pd.read_excel('C:\\Users\\raque\\OneDrive\\Documents\\TFG\\Datos\\radiomicas.xlsx')

# Leer el DataFrame de datos clínicos
clinicas = pd.read_excel('C:\\Users\\raque\\OneDrive\\Documents\\TFG\\Datos\\clinicas_faltantes.xlsx')

## Selección de observaciones

In [4]:
radiomicas_filtradas = radiomicas[radiomicas['patient_id'].isin(clinicas['patient_id'])]
clinicas_filtradas = clinicas[clinicas['patient_id'].isin(radiomicas['patient_id'])]

In [5]:
print(radiomicas_filtradas.shape)
print(clinicas_filtradas.shape)

(323, 111)
(323, 11)


## Imputación de valores faltantes

A continuación, vamos a estudiar los valores faltantes e imputarlo.

In [6]:
clinicas_filtradas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323 entries, 0 to 4628
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   patient_id            323 non-null    object 
 1   age                   323 non-null    int64  
 2   psas_0_total          307 non-null    float64
 3   lesions_0_pi_rads     316 non-null    float64
 4   lesions_0_gleason1.1  314 non-null    float64
 5   lesions_0_gleason2.1  314 non-null    float64
 6   ISUP                  314 non-null    float64
 7   TZ                    323 non-null    bool   
 8   PZ                    323 non-null    bool   
 9   CZ                    323 non-null    bool   
 10  AS                    323 non-null    bool   
dtypes: bool(4), float64(5), int64(1), object(1)
memory usage: 21.4+ KB


In [7]:
nan_counts = clinicas_filtradas.isna().sum()
nan_counts

patient_id               0
age                      0
psas_0_total            16
lesions_0_pi_rads        7
lesions_0_gleason1.1     9
lesions_0_gleason2.1     9
ISUP                     9
TZ                       0
PZ                       0
CZ                       0
AS                       0
dtype: int64

Calculamos la media de las variables numéricas para imputar los faltantes por ese valor. Sin embargo, dado que las últimas cuatro variables `lesions_0_pi_rads`, `lesions_0_gleason1.1`, `lesions_0_gleason2.1`, e `ISUP` son probabilidades, se han de redondear esas medias para que sean enteros.

In [8]:
mean= clinicas_filtradas.iloc[:, 1:7].mean()
mean.loc[2:] = mean.iloc[2:].round()
mean

age                     65.139319
psas_0_total            11.269524
lesions_0_pi_rads        4.000000
lesions_0_gleason1.1     3.000000
lesions_0_gleason2.1     4.000000
ISUP                     2.000000
dtype: float64

In [9]:
clinicas_filtradas= clinicas_filtradas.fillna(mean)

In [10]:
nan_cts = clinicas_filtradas.isna().sum()
nan_cts

patient_id              0
age                     0
psas_0_total            0
lesions_0_pi_rads       0
lesions_0_gleason1.1    0
lesions_0_gleason2.1    0
ISUP                    0
TZ                      0
PZ                      0
CZ                      0
AS                      0
dtype: int64

In [11]:
# Convertir las variables a enteros
clinicas_filtradas[['lesions_0_pi_rads', 'lesions_0_gleason1.1', 'lesions_0_gleason2.1', 'ISUP']] = clinicas_filtradas[['lesions_0_pi_rads', 'lesions_0_gleason1.1', 'lesions_0_gleason2.1', 'ISUP']].astype(int)

In [12]:
clinicas_filtradas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323 entries, 0 to 4628
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   patient_id            323 non-null    object 
 1   age                   323 non-null    int64  
 2   psas_0_total          323 non-null    float64
 3   lesions_0_pi_rads     323 non-null    int32  
 4   lesions_0_gleason1.1  323 non-null    int32  
 5   lesions_0_gleason2.1  323 non-null    int32  
 6   ISUP                  323 non-null    int32  
 7   TZ                    323 non-null    bool   
 8   PZ                    323 non-null    bool   
 9   CZ                    323 non-null    bool   
 10  AS                    323 non-null    bool   
dtypes: bool(4), float64(1), int32(4), int64(1), object(1)
memory usage: 16.4+ KB


## Exportación df

In [13]:
path_radiomicas_filtradas = 'C:\\Users\\raque\\OneDrive\\Documents\\TFG\\Datos\\radiomicas_filtradas.xlsx'
radiomicas_filtradas.to_excel(path_radiomicas_filtradas, index=False)

path_clinicas_filtradas = 'C:\\Users\\raque\\OneDrive\\Documents\\TFG\\Datos\\clinicas_filtradas.xlsx'
clinicas_filtradas.to_excel(path_clinicas_filtradas, index=False)